In [ ]:
import pandas as pd

data = pd.read_csv("youtube_data.csv")
data.describe()

# You only need the other line if you find empty cells in the table
# data = data.dropna(axis=0)
# There are also other methods with dealing with missing data - imputing (putting in averages) and removing the entire colummn

,duration,bitrate,bitrate(video),height,width,frame rate,frame rate(est.),views,likes,comments
count,17589.000000,17589.000000,17589.000000,17589.000000,17589.000000,17589.000000,17589.000000,1.758900e+04,17589.000000,17589.000000
mean,241.551936,1271.354369,1150.418443,766.781170,504.591961,26.467639,9.471172,6.894158e+04,208.862641,12.899312
std,493.026994,1375.359875,1351.800202,467.289304,262.727746,6.039748,13.253197,3.537491e+06,8477.504735,225.839378
min,1.000000,0.000000,0.000000,108.000000,88.000000,3.750000,0.000000,0.000000e+00,0.000000,0.000000
25%,51.000000,437.000000,326.000000,426.000000,320.000000,25.000000,0.000000,3.800000e+01,0.000000,0.000000
50%,135.000000,743.000000,632.000000,640.000000,480.000000,29.970000,0.000000,1.520000e+02,1.000000,0.000000
75%,268.000000,1293.000000,1184.000000,960.000000,720.000000,29.970000,25.000000,8.000000e+02,4.000000,1.000000
max,25845.000000,22421.000000,22229.000000,2592.000000,1944.000000,59.080000,30.000000,4.103849e+08,836981.000000,16634.000000


In [ ]:
y = data.views
features = ['duration','bitrate','frame rate','likes','comments']
X = data[features]

# break up the data into training data and testing data
# remember - the random state number isn't important other than it is used to make reproducible results
# if other functions call for a random_state number, they do not have to match, they only have to remain fixed for reproducible results
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X,y,random_state = 1)

# Compare the preformance of different ML models



# First, the Decision Tree without leaf optimization
from sklearn.tree import DecisionTreeRegressor
model1 = DecisionTreeRegressor(random_state = 2)
model1.fit(train_X, train_y)

from sklearn.metrics import mean_absolute_error

print("The Mean Absolute Error for the Decision Tree ML model without leaf optimization is",mean_absolute_error(model1.predict(val_X),val_y))

#Second, the Decision Tree with leaf optimization
def get_mae(leaves, train_X, val_X, train_y, val_y):
  model2 = DecisionTreeRegressor(max_leaf_nodes=leaves, random_state=55)
  model2.fit(train_X, train_y)
  return mean_absolute_error(model2.predict(val_X), val_y)

leaves = [50,500,5000]
lowest = get_mae(5,train_X, val_X, train_y, val_y)
for i in leaves:
  if get_mae(i,train_X, val_X, train_y, val_y)<lowest:
    lowest = get_mae(i,train_X, val_X, train_y, val_y)
print("The Mean Absolute Error for the Decision Tree ML model with leaf optimization is",lowest)

#Third, the Random Forest
from sklearn.ensemble import RandomForestRegressor
model3 = RandomForestRegressor(n_estimators=50, random_state = 5)
model3.fit(train_X, train_y)
print("The Mean Absolute Error for the Random Tree ML model is",mean_absolute_error(model3.predict(val_X),val_y))

#Fourth XGBoost
from xgboost import XGBRegressor
model4 = XGBRegressor(n_estimators = 1000, learning_rate = 0.05, verbosity = 1, early_stopping_rounds=10)
model4.fit(train_X, train_y,eval_set=[(val_X, val_y)])
print("The Mean Absolute Error for the XGBoost model is", mean_absolute_error(model4.predict(val_X), val_y))




The Mean Absolute Error for the Decision Tree ML model without leaf optimization is 85031.5957253297
The Mean Absolute Error for the Decision Tree ML model with leaf optimization is 83774.18508155244
The Mean Absolute Error for the Random Tree ML model is 53807.63474279435
[0]	validation_0-rmse:2063600.03748
[1]	validation_0-rmse:2071173.08195
[2]	validation_0-rmse:2087323.13217
[3]	validation_0-rmse:2111020.71740
[4]	validation_0-rmse:2141212.93666
[5]	validation_0-rmse:2176943.92359
[6]	validation_0-rmse:2217285.59263
[7]	validation_0-rmse:2261438.91323
[8]	validation_0-rmse:2309296.99148
[9]	validation_0-rmse:2322661.00655
[10]	validation_0-rmse:2335421.06136
The Mean Absolute Error for the XGBoost model is 118114.9921875
